In [1]:
import pandas as pd
import glob
from googletrans import Translator
from google_trans_new import google_translator  
import emoji
import re
from langdetect import detect
from numba import jit
from concurrent.futures import ThreadPoolExecutor

In [2]:
directory = 'targeted'
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)  # Show all rows
#pd.set_option('display.max_columns', None)

In [3]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001F680-\U0001F6FF"  # CJK Unified Ideographs Extension B
                           u"\U0001F600-\U0001F64F"  # CJK Unified Ideographs
                           u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols and Pictographs"
                           u"\U0001F000-\U0001F02F"  # Mahjong Tiles
                           u"\U0001F004-\U0001F0CF"  # Domino Tiles
                           u"\U0001F110-\U0001F251"  # Playing Cards
                           u"\U0001F600-\U0001F64F"  # Emoticons (faces)
                           u"\U0001F910-\U0001F91F"  # Emojis (animal faces)
                           u"\U0001F920-\U0001F927"  # Emojis (human faces)
                           u"\U0001F930-\U0001F939"  # Emojis (clothing)
                           u"\U0001F950-\U0001F95F"  # Emojis (food)
                           u"\U0001F980-\U0001F991"  # Emojis (objects)
                           u"\U0001F9C0-\U0001F9C0"  # Emojis (symbols)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)



In [4]:
#attempting to test for one spreadsheet to determine run time on singular translation output


t = Translator()

translations = []
test_file = glob.glob(directory + "/targeted.xlsx")
test_list = []
for file in test_file:
    df = pd.read_excel(file, sheet_name='Tweets')
    test_list.append(df)


# Concatenate the DataFrames in test_list into a single DataFrame
combined_df = pd.concat(test_list, axis=0, ignore_index=True)
combined_df = combined_df.drop(['Binders', 'Tweet value', 'Likes count', 'Retweet count'], axis = 1)
combined_df['Tweet'] = combined_df['Tweet'].str.replace("’", "'")
combined_df['Tweet'] = combined_df['Tweet'].apply(remove_emojis)
for text in combined_df['Tweet']:
    eng = str(1)
    if not text.isascii():
        eng = str(0)
        translations.append(eng)
    else:
        translations.append(eng)
# Add the translations to a new column in the DataFrame

combined_df['Translated_Tweet'] = translations



In [36]:
combined_df



,User,Username,UTC Date,Tweet,Permalink,Translated_Tweet
0,Mr Gumede,@MrGumede195,2023-02-04 14:44:19,RT @VeengasJ: A video of yesterday incident #Hafeezbaloch and his family were targeted by security agencies.\nCitizen captured the incident. A video explained everything how they are treating Baloch and ruling parties either in federal government or Sindh government both are silent. https://t.co/oOCU2B3Ra4,https://www.twitter.com/user/status/1621882346872913921,1
1,#TrudeauMustGo - Aldo,@aldoruscio,2023-02-04 14:44:15,RT @Concern70732755: If a well regarded media source openly condemns Trudeau's planned targeted suppression &amp; covert censorship that also provides content control who doesn't our MSM make any comments ? \nEnough said -truth &amp; objectivity are already lost !! https://t.co/If5gNYCIqQ,https://www.twitter.com/user/status/1621882331282677762,1
2,PennySue,@BoomerSooner59,2023-02-04 14:44:11,"RT @_LibertyLady: “Eunice Dwumfour, a 30-year-old New Jersey councilwoman, was found dead outside her Sayreville home Wednesday evening in what officials believe was a “targeted” attack, authorities say.” \n\nWhy isn't #JusticeForEunice trending? \n\nhttps://t.co/3uUPEhYQ0j",https://www.twitter.com/user/status/1621882315109502981,0
3,ShahDadBaloch,@ShahDad49238732,2023-02-04 14:44:11,"RT @VeengasJ: Saeeda Zehri narrated how security agencies targeted and attacked her uncle Hafeez Zehri, mother and family members who went to Karachi Central Jail for the release of Hafeez.\nShe shows bloodstained on her shawl how family members were attacked.\n""What does state want,"" she asks. https://t.co/MPwnURGH2k",https://www.twitter.com/user/status/1621882312970440711,1
4,honey💗,@_highsolis,2023-02-04 14:44:10,RT @J2MTargeted: We all die eventually\nI rather die with integrity\nIt's hard but I do love my enemies\nNothing is more important than family\nWe as a community\nNeed to have unity\nLife isn't simple\nBeing a #TargetedIndividual,https://www.twitter.com/user/status/1621882311321993217,1
...,...,...,...,...,...,...
9995,Justa Notherguy 🇺🇸,@CjCohan,2023-02-04 06:43:27,RT @Joshuajered: BREAKING: New Jersey Republican Councilwoman Eunice Dwumfour was assassinated outside her home. This was the scene overnight in Sayreville where her white SUV crashed after gunshots rang out. Police say it appears she was targeted and are still searching for several suspects. https://t.co/O01Scjcmpm,https://www.twitter.com/user/status/1621761334332252160,1
9996,That farmer boy,@Farmexecutive,2023-02-04 06:43:25,"So a presidential candidate assumed a policy for over 200m people was targeted at him,God how on earth did we get here ?",https://www.twitter.com/user/status/1621761325813596161,1
9997,Aashish,@Aashish__jha,2023-02-04 06:43:24,"RT @MCStanOfficial_: Stan was the best performer in yesterday's task NO CAP \nThey targeted him the most, bhot yedi chaalein ki but APPA Ek inch nai hila spot se. HAQ HAI STANNY KO #MCStan𓃵 #BBKingMCStan no one deserves to pick the trophy like this guy does #biggboss16",https://www.twitter.com/user/status/1621761322491977728,0
9998,MJ,@MJ_007Club,2023-02-04 06:43:23,"Ilhan Omar starts playing “Muslim” victim card for her ouster over Anti-Israel comments.\n\nThey just want to stay relevant for all the wrong reasons.\n\n“I'm a Muslim. I'm an immigrant. Is anyone surprised that I'm being targeted? I'm an American”, she said",https://www.twitter.com/user/status/1621761315671883776,0


In [5]:
import time

import numba

mask = combined_df['Translated_Tweet'].str.contains('0', case=False)
#combined_df = combined_df.drop(['Binders', 'Tweet value', 'Likes count', 'Retweet count'], axis = 1)

#combined_df.to_csv('combined_df_csv.csv')

# Use the mask to filter rows that match the condition
filtered_rows = combined_df[mask]
filtered_rows['Tweet'] = filtered_rows['Tweet'].str.replace("’", "'")
filtered_rows['Tweet'] = filtered_rows['Tweet'].apply(remove_emojis)

filtered_rows.to_csv('filtered.csv')

# Initialize the translator
t = Translator()

@numba.jit
# Define a function to translate text
def translate_text(text):
    try:
        translated_text = t.translate(text, dest='en').text
        return translated_text
    except Exception as e:
        print(f"Error translating: {e}")
        return None

# Load your DataFrame
df = pd.read_csv('filtered.csv')

# Extract the 'Tweet' column
texts = df['Tweet'].tolist()

translations = []

# Translate the texts concurrently using ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    results = list(executor.map(translate_text, texts))
    time.sleep(1)

for result in results:
    if result is not None:
        translations.append(result)

# Add the translations to a new column in the DataFrame
df['Translated'] = translations

/var/folders/4_/rf44p2zs3ld9skyp7yxhcm8r0000gn/T/ipykernel_49278/2084300875.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Tweet'] = filtered_rows['Tweet'].str.replace("’", "'")
/var/folders/4_/rf44p2zs3ld9skyp7yxhcm8r0000gn/T/ipykernel_49278/2084300875.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Tweet'] = filtered_rows['Tweet'].apply(remove_emojis)


Error translating: <ConnectionTerminated error_code:ErrorCodes.PROTOCOL_ERROR, last_stream_id:17, additional_data:696e76616c69645f6e65775f73747265616d5f69>
Error translating: Invalid input ConnectionInputs.RECV_HEADERS in state ConnectionState.CLOSED
Error translating: Invalid input ConnectionInputs.RECV_DATA in state ConnectionState.CLOSED
Error translating: Invalid input ConnectionInputs.RECV_DATA in state ConnectionState.CLOSED
Error translating: Invalid input ConnectionInputs.RECV_PING in state ConnectionState.CLOSED
Error translating: Invalid input ConnectionInputs.RECV_HEADERS in state ConnectionState.CLOSED
Error translating: Invalid input ConnectionInputs.RECV_DATA in state ConnectionState.CLOSED
Error translating: Invalid input ConnectionInputs.RECV_DATA in state ConnectionState.CLOSED
Error translating: Invalid input ConnectionInputs.RECV_DATA in state ConnectionState.CLOSED
Error translating: Invalid input ConnectionInputs.RECV_HEADERS in state ConnectionState.CLOSED
Error t

In [37]:
# Create a boolean mask by applying the condition to the 'Tweet' column

mask = combined_df['Translated_Tweet'].str.contains('0', case=False)
#combined_df = combined_df.drop(['Binders', 'Tweet value', 'Likes count', 'Retweet count'], axis = 1)

#combined_df.to_csv('combined_df_csv.csv')

# Use the mask to filter rows that match the condition
filtered_rows = combined_df[mask]
filtered_rows['Tweet'] = filtered_rows['Tweet'].str.replace("’", "'")
filtered_rows['Tweet'] = filtered_rows['Tweet'].apply(remove_emojis)

filtered_rows.to_csv('filtered.csv')

df = pd.read_csv('filtered.csv')
df
# Display the filtered DataFrame
#df = pd.DataFrame(filtered_rows, columns = ['User',	'Username',	'UTC Date',	'Tweet'	,'Binders',	'Permalink',	'Retweet count',	'Likes count',	'Tweet value',	'Translated_Tweet'])




/var/folders/4_/rf44p2zs3ld9skyp7yxhcm8r0000gn/T/ipykernel_31956/2398825684.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Tweet'] = filtered_rows['Tweet'].str.replace("’", "'")
/var/folders/4_/rf44p2zs3ld9skyp7yxhcm8r0000gn/T/ipykernel_31956/2398825684.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Tweet'] = filtered_rows['Tweet'].apply(remove_emojis)


,Unnamed: 0,User,Username,UTC Date,Tweet,Permalink,Translated_Tweet
0,2,PennySue,@BoomerSooner59,2023-02-04 14:44:11,"RT @_LibertyLady: “Eunice Dwumfour, a 30-year-old New Jersey councilwoman, was found dead outside her Sayreville home Wednesday evening in what officials believe was a “targeted” attack, authorities say.” \n\nWhy isn't #JusticeForEunice trending? \n\nhttps://t.co/3uUPEhYQ0j",https://www.twitter.com/user/status/1621882315109502981,0
1,7,Daniel Kemp,@danielkemp6,2023-02-04 14:44:09,"The British ‘over-the-horizon' radar stations on Cyprus are being targeted by Russia and Syria. World safety depends on the terrorist-inspired opposition being obliterated. \n\nMy Truth, Your Lies —https://t.co/rZn1erDuf8 https://t.co/R6utRQ2FRo",https://www.twitter.com/user/status/1621882305160613888,0
2,9,The Masstronaut,@TMasstronaut,2023-02-04 14:43:57,"RT @BecketAdams: you got Rep. Ocasio-Cortez out here invoking “stochastic terrorism” while protesting Rep. Omar's removal from the House Foreign Affairs Committee. meanwhile, a black New Jersey GOP councilwoman was shot to death in what law enforcement believe was a targeted assassination.",https://www.twitter.com/user/status/1621882255730741248,0
3,10,Goldens Rule,@jamesbranch3,2023-02-04 14:43:56,"RT @BecketAdams: you got Rep. Ocasio-Cortez out here invoking “stochastic terrorism” while protesting Rep. Omar's removal from the House Foreign Affairs Committee. meanwhile, a black New Jersey GOP councilwoman was shot to death in what law enforcement believe was a targeted assassination.",https://www.twitter.com/user/status/1621882251675074560,0
4,16,CSullyA,@CASABC12321,2023-02-04 14:43:33,"RT @BecketAdams: you got Rep. Ocasio-Cortez out here invoking “stochastic terrorism” while protesting Rep. Omar's removal from the House Foreign Affairs Committee. meanwhile, a black New Jersey GOP councilwoman was shot to death in what law enforcement believe was a targeted assassination.",https://www.twitter.com/user/status/1621882155667423233,0
...,...,...,...,...,...,...,...
2239,9985,Jake Jacobs,@JakeJacobs1776,2023-02-04 06:44:09,RT @ClimateDepot: YOUR Freedom of Movement targeted: Scottish govt declares era of ‘unconstrained growth in private car use is well &amp; truly over' – Aims to ‘discourage car use' by reducing ‘need to travel unsustainably' to meet Net Zero goals | Climate Depot https://t.co/yyYv378alJ,https://www.twitter.com/user/status/1621761509826203649,0
2240,9990,Special Aurora Borealis 😌,@ButterflyMagick,2023-02-04 06:43:43,"RT @VirkSh786: Since past days there have been a lot of propaganda regarding “days spent in jails” by PMLN.\n\nThey have targeted Fawad, Gill, Swati but never mentioned that they were taken to jail for frivolous cases, while PMLN went for corruption charges. And they have not seen any torture.",https://www.twitter.com/user/status/1621761400530997248,0
2241,9997,Aashish,@Aashish__jha,2023-02-04 06:43:24,"RT @MCStanOfficial_: Stan was the best performer in yesterday's task NO CAP \nThey targeted him the most, bhot yedi chaalein ki but APPA Ek inch nai hila spot se. HAQ HAI STANNY KO #MCStan𓃵 #BBKingMCStan no one deserves to pick the trophy like this guy does #biggboss16",https://www.twitter.com/user/status/1621761322491977728,0
2242,9998,MJ,@MJ_007Club,2023-02-04 06:43:23,"Ilhan Omar starts playing “Muslim” victim card for her ouster over Anti-Israel comments.\n\nThey just want to stay relevant for all the wrong reasons.\n\n“I'm a Muslim. I'm an immigrant. Is anyone surprised that I'm being targeted? I'm an American”, she said",https://www.twitter.com/user/status/1621761315671883776,0


In [23]:

t = Translator()

translations = []

# Iterate through the 'Tweet' column and translate non-ASCII texts
for text in df['Tweet']:
    try:
        translated_text = t.translate(text, dest='en').text
        #print(translated_text)
        translations.append(translated_text)
        time.sleep(1)
        #print(translations)
    except Exception as e:
        print(f"Error translating: {e}")

# Add the translations to a new column in the DataFrame
df['Translated'] = translations

In [ ]:
combined_df

In [24]:
df

,Unnamed: 0,User,Username,UTC Date,Tweet,Permalink,Translated_Tweet,Translated
0,1,#TrudeauMustGo - Aldo,@aldoruscio,2023-02-04 14:44:15,RT @Concern70732755: If a well regarded media source openly condemns Trudeau's planned targeted suppression &amp; covert censorship that also provides content control who doesn't our MSM make any comments ? \nEnough said -truth &amp; objectivity are already lost !! https://t.co/If5gNYCIqQ,https://www.twitter.com/user/status/1621882331282677762,0,RT @Concern70732755: If a well regarded media source openly condemns Trudeau's planned targeted suppression &amp; covert censorship that also provides content control who doesn't our MSM make any comments ? \nEnough said -truth &amp; objectivity are already lost !! https://t.co/If5gNYCIqQ
1,2,PennySue,@BoomerSooner59,2023-02-04 14:44:11,"RT @_LibertyLady: “Eunice Dwumfour, a 30-year-old New Jersey councilwoman, was found dead outside her Sayreville home Wednesday evening in what officials believe was a “targeted” attack, authorities say.” \n\nWhy isn't #JusticeForEunice trending? \n\nhttps://t.co/3uUPEhYQ0j",https://www.twitter.com/user/status/1621882315109502981,0,"RT @_LibertyLady: “Eunice Dwumfour, a 30-year-old New Jersey councilwoman, was found dead outside her Sayreville home Wednesday evening in what officials believe was a “targeted” attack, authorities say.” \n\nWhy isn't #JusticeForEunice trending? \n\nhttps://t.co/3uUPEhYQ0j"
2,7,Daniel Kemp,@danielkemp6,2023-02-04 14:44:09,"The British ‘over-the-horizon' radar stations on Cyprus are being targeted by Russia and Syria. World safety depends on the terrorist-inspired opposition being obliterated. \n\nMy Truth, Your Lies —https://t.co/rZn1erDuf8 https://t.co/R6utRQ2FRo",https://www.twitter.com/user/status/1621882305160613888,0,"The British ‘over-the-horizon' radar stations on Cyprus are being targeted by Russia and Syria. World safety depends on the terrorist-inspired opposition being obliterated. \n\nMy Truth, Your Lies —https://t.co/rZn1erDuf8 https://t.co/R6utRQ2FRo"
3,9,The Masstronaut,@TMasstronaut,2023-02-04 14:43:57,"RT @BecketAdams: you got Rep. Ocasio-Cortez out here invoking “stochastic terrorism” while protesting Rep. Omar's removal from the House Foreign Affairs Committee. meanwhile, a black New Jersey GOP councilwoman was shot to death in what law enforcement believe was a targeted assassination.",https://www.twitter.com/user/status/1621882255730741248,0,"RT @BecketAdams: you got Rep. Ocasio-Cortez out here invoking “stochastic terrorism” while protesting Rep. Omar's removal from the House Foreign Affairs Committee. meanwhile, a black New Jersey GOP councilwoman was shot to death in what law enforcement believe was a targeted assassination."
4,10,Goldens Rule,@jamesbranch3,2023-02-04 14:43:56,"RT @BecketAdams: you got Rep. Ocasio-Cortez out here invoking “stochastic terrorism” while protesting Rep. Omar's removal from the House Foreign Affairs Committee. meanwhile, a black New Jersey GOP councilwoman was shot to death in what law enforcement believe was a targeted assassination.",https://www.twitter.com/user/status/1621882251675074560,0,"RT @BecketAdams: you got Rep. Ocasio-Cortez out here invoking “stochastic terrorism” while protesting Rep. Omar's removal from the House Foreign Affairs Committee. meanwhile, a black New Jersey GOP councilwoman was shot to death in what law enforcement believe was a targeted assassination."
...,...,...,...,...,...,...,...,...
3688,9990,Special Aurora Borealis 😌,@ButterflyMagick,2023-02-04 06:43:43,"RT @VirkSh786: Since past days there have been a lot of propaganda regarding “days spent in jails” by PMLN.\n\nThey have targeted Fawad, Gill, Swati but never mentioned that they were taken to jail for frivolous cases, while PMLN went for corruption charges. And they have not seen any torture.",https://www.twitter.com/user/status/1621761400530997248,0,"RT @VirkSh786: Since past days there have been a lot of propaganda regarding “days spent in jails” by 

In [ ]:
combined_df.iloc[12, :]

In [19]:
#merges all targeted files into one dataframe 
directory = 'targeted'
pd.set_option('display.max_colwidth', None)

#glob searches for files that match a specific pattern or name
file_list = glob.glob(directory + "/*.xlsx")


excl_list = []
for file in file_list:
    excl_list.append(pd.read_excel(str(file), sheet_name = 'Tweets'))
#create the merged dataset with all targeted data
excl_merged = pd.concat(excl_list, ignore_index=True)

for text in excl_merged['Tweet']:
    eng = str(1)
    if not text.isascii():
        eng = str(0)
        translations.append(eng)
    else:
        translations.append(eng)
# Add the translations to a new column in the DataFrame

excl_merged['Translated_Tweet'] = translations


In [20]:
#drops unnecessary columns 


excl_merged.drop(['Binders', 'Tweet value', 'Likes count', 'Retweet count'], axis = 1)


,User,Username,UTC Date,Tweet,Permalink
0,Gary Hill,@hillg367,2023-01-30 19:39:18,"RT @davenewworld_2: 4 white supremacists in Washington state were sentenced to 28 to 48 months for a hate crime after they entered a bar and assaulted the DJ, Tyrone Smith, while calling him slurs. They targeted a Black man on ""Martyr's Day"" to ""honor"" a white supremacist who died in the 1980s. https://t.co/O3ojTWDpVL",https://www.twitter.com/user/status/1620144641940258816
1,خالد الشعيبي,@qwe716128968,2023-01-30 19:39:16,The US directly targeted all economic sectors in #Yemen and looted all the revenues leaving the state employees with nothing to live off of\n#USAEnemyOfPeace\n#امريكا_عدو_السلام,https://www.twitter.com/user/status/1620144636416380928
2,ampk.95,@ampk_95,2023-01-30 19:39:16,@mccnshyne @akitoshinonome Not really an otome at all. More just like you learn lore about the characters and listen to all the songs they got released. Theres a game where you can collect their cards. Technically targeted for women but like you just kinda hang out with them as they info dump their issues,https://www.twitter.com/user/status/1620144635808223233
3,عبدالله,@bdllh16716948,2023-01-30 19:39:15,Every aspect of life in #Yemen was targeted by the US-Saudi aggression even chicken farms and horse stables were not left untouched\n#USAEnemyOfPeace\n#امريكا_عدو_السلام\nhttps://t.co/ieboNZTpfD,https://www.twitter.com/user/status/1620144629135069184
4,llwry,@LLLwry,2023-01-30 19:39:14,"RT @davenewworld_2: 4 white supremacists in Washington state were sentenced to 28 to 48 months for a hate crime after they entered a bar and assaulted the DJ, Tyrone Smith, while calling him slurs. They targeted a Black man on ""Martyr's Day"" to ""honor"" a white supremacist who died in the 1980s. https://t.co/O3ojTWDpVL",https://www.twitter.com/user/status/1620144627256025088
...,...,...,...,...,...
459854,Jamie Chirps 🦜🇨🇦,@JamieLC6,2023-01-19 23:21:16,RT @Roman_Baber: ICYMI. The same Canadian Armed Forces team that engaged in propaganda against Canadians at the start of the Pandemic has targeted Canadians again. \n\n@AnitaAnandMP &amp; @JustinTrudeau are spreading propaganda and using the @CanadianForces against Canadians. https://t.co/doJZQXCdns,https://www.twitter.com/user/status/1616214235918356481
459855,UnacceptableSilverFox 🍁,@Cdn_Eliana,2023-01-19 23:21:08,RT @Roman_Baber: ICYMI. The same Canadian Armed Forces team that engaged in propaganda against Canadians at the start of the Pandemic has targeted Canadians again. \n\n@AnitaAnandMP &amp; @JustinTrudeau are spreading propaganda and using the @CanadianForces against Canadians. https://t.co/doJZQXCdns,https://www.twitter.com/user/status/1616214205010694145
459856,SteelerMoo,@FatherMoo,2023-01-19 23:21:06,"RT @ESPNRingside: Stephen Fulton and Naoya Inoue have agreed to terms for a unified 122-pound title fight in the spring in Japan, sources told @MikeCoppinger.\n\nThe fight for Fulton's WBC and WBO titles is being targeted for May, though no date is set yet. The bout will be streamed on @ESPNPlus. https://t.co/e0TtMsTGoP",https://www.twitter.com/user/status/1616214194184937474
459857,fiVth,@VthpoetV,2023-01-19 23:21:06,"RT @TasetiReloaded: After the expulsion of the Moors, the Christian church SANCTIONED the enslavement of Africans. Africans were a TARGETED population. Europeans went to Africa with PREMEDITATION to enslave. Inconsequential if some Africans were sellers b/c Euros would enslave Africans thru FORCE.",https://www.twitter.com/user/status/1616214192918249472


In [ ]:
#excl_merged['Tweet']

t = Translator()

translations = []

# Iterate through the 'Tweet' column and translate non-ASCII texts
for text in excl_merged['Tweet']:
    if emoji.analyze(text) == False or not text.isascii() :
        try:
            translated_text = t.translate(text, dest='en').text
            #print(translated_text)
            translations.append(translated_text)
            print(translations)
        except Exception as e:
            print(f"Error translating: {e}")

# Add the translations to a new column in the DataFrame
excl_merged['Translated_Tweet'] = translations

#THIS TAKES TOO LONG NEED TO IMPLEMENT ANOTHER SOLUTION!!!



In [ ]:
excl_merged['Translated_Tweet']

In [ ]:
excl_merged.to_csv('excl_merged.csv')

In [ ]:
df = pd.read_csv('excl_merged.csv')
df

In [ ]:
df['Tweet']